Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [47]:
#%%writefile publications.py
import numpy as np
import pandas as pd
import re
import requests
import warnings
import time
import sys
from IPython.display import display, HTML
import tabulate #sudo pip3 install tabulate
import utilities as ut
from cmdlike import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)
def update_column(ds,column,update=False):
    update_column=True
    if column in ds:
        if ds[column] and not update: #it update=True willreplace value
            update_column=False
    return update_column

def _get_doi(surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        '''
        Search doi from http://search.crossref.org/ 
        '''
        doi={}
        search=''
        if surname:
            search=surname
        if title:
            if len(search)>0:
                search=search+', '+title
        if other:
            if len(search)>0:
                search=search+', '+other
                
        r=requests.get('http://search.crossref.org/?q=%s' %search)
        urldoi='http://dx.doi.org/'
        doitmp=''
        if len(r.text.split(urldoi))>1:
            doitmp=r.text.split(urldoi)[1].split("\'>")[0].replace('&lt;','<').replace('&gt;','>')
        #check doi is right by searching for all words in doi -> output title
        if doitmp:
            json='https://api.crossref.org/v1/works/'
            rr=requests.get( json+urldoi+doitmp )
            if rr.status_code==200:
                if 'message' in rr.json():
                    chktitle = re.sub(r"\$.*?\$","",title) # better remove all math expressions
                    chktitle = re.sub(r"[^a-zA-Z0-9 ]", " ", chktitle).split(' ')
                    if chktitle:
                        if not -1 in [(rr.json()["message"]['title'][0]).find(w)  for w in chktitle]:
                            doi=rr.json()["message"]
                        
        return doi
    
def _get_impact_factor_from_journal_name(journal_name='Physical Review D'):
    '''
      For the input Journal name obtain
      the pandas DataFrame with Years and IF as columns
    '''
    q=journal_name.lower().replace(' ','-')
    URL='http://www.journal-database.com/journal/%s.html' %q        
    r = requests.get(URL)
    return ut.html_to_DataFrame(r.content)
    #UDPATE in repo
    
def _get_quartil(issn='1550-7998',journal_hindex=False):
    URL='http://www.scimagojr.com/journalsearch.php?q='
    r=requests.get('%s%s' %(URL,issn))
    result=r.text.split('href="journalsearch.php?q=')
    quartil='';hindex_journal=''
    if type(result)==list and len(result)==3:
        id_jour=result[-1].split('">')[0]
        if id_jour:
            rr=requests.get('%s%s' %(URL,id_jour))
            quartil=grep('^\s+var dataquartiles = ',rr.text).split(';')
            quartil.remove('')
            if type(quartil)==list and len(quartil)>0:
                quartil=quartil[-1].replace('"','')
            hindex=rr.text.split('<div class="hindexnumber">')
            if type(hindex)==list and len(hindex)==2:
                hindex_journal=hindex[1].split('</div>')[0]
    if journal_hindex:
        return quartil,hindex_journal
    else:
        return quartil    

class publications(object):
    '''Add Generic publication data'''
    pass
class articles(publications):
    '''Read csv file exported by Google Scholar Citations profile and
       automatically add informations about:
       DOI: from title and author
       Journal title: from DOI
       ISSN of Jornal
       Impact factor of journal
       Institution_Authors: requires Data Base
       '''
    journal=pd.Series()
    columns=pd.Series({'Full_Name':'Full_Name','Author_Names':'Author_Names','Control':'Control',\
                      'Institution_Authors':'Institution_Authors','Institution_Group':'Institution_Group'})

    def __init__(self,csv_file='citations.csv',citations_file=None,authors_file=None,group_file=None):
        #DEBUG: check file
        self.articles=pd.read_csv(csv_file).fillna('')
        #Fix problem with column Authors
        if self.articles.shape[0]>0 and self.articles.columns[0].find('Authors')>-1:
            self.articles=self.articles.rename(columns={self.articles.columns[0]:'Authors'})
        if citations_file:
            #DEBUG: check file
            self.citations=pd.read_csv(citations_file).fillna('')
            
        if authors_file:
            #DEBUG: chek is file exists,
            self.institution_authors=pd.read_json(authors_file).fillna('')
            
        if group_file:
            #DEBUG: chek is file exists,
            self.institution_group=pd.read_json(group_file).fillna('')
            
    def add_institution_author(self):
        #DEBUG: Check and load file
        full_name=input('Full name: ')
        #DEBUG: Ask for more author names and update author names separated by semicolons
        author_names=input('Author names\n(Example: Juan Perez; J. Perez; J. Pérez):\n')
        #DEBUG: default to 0
        control=input('Additional identitication number: ') 
        
        #DEBUG: if not file creates it
        iffile=False
        if not iffile:
            self.institution_authors=pd.DataFrame()
            
        self.institution_authors=self.institution_authors.append({self.columns.Full_Name:full_name,\
                                                              self.columns.Author_Names:author_names,\
                                                              self.columns.Control:control},ignore_index=True)
        self.institution_authors.to_json('authors.json')
        
    def add_institution_group(self):
        wrn='Author not found'
        search_query=input('Find author by first name and first surname\n(Example: Juan Pérez);\n')
        sql=search_query.split(' ')
        #DEBUG: low_case
        if len(sql)>0:
            sql_name=sql[0]
            if len(sql)>1:
                sql_surname=sql[1]
            else:
                sql_surname=''
            
            author_match=self.institution_authors[\
                             np.logical_and( self.institution_authors.Full_Name.str.contains(sql_name),\
                                             self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                author_match=self.institution_authors[\
                             np.logical_or( self.institution_authors.Full_Name.str.contains(sql_name),\
                                            self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                warnings.warn(wrn)
            else:    
                author_match=author_match.reset_index(drop=True)
                author_match['Number']=author_match.index
                if author_match.shape[0]==1:
                    author_Series=author_match.ix[0]
                    #print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                else:
                    print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                    ai=input('Give Number of line:')
                    author_Series=author_match.ix[ai]
                Group=input('Group name for\n%s\n(Example: IA: Inteligencia Artificial)' %author_Series.Full_Name)
                #DEBUG: if not file creates it
                iffile=False
                if not iffile:
                    self.institution_group=pd.DataFrame()  
                    self.institution_group=self.institution_group.append({self.columns.Full_Name:author_Series.Full_Name,\
                                                                  self.columns.Institution_Group:Group},ignore_index=True)
                    self.institution_group.to_json('groups.json')
        else:
            warnings.warn(wrn)

        
        
    def get_doi(self,surname='Florez',\
                     title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        return _get_doi(surname='Florez',\
                        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other='')
    
    def get_IF(self,journal_name='Physical Review D'):
        return _get_impact_factor_from_journal_name(journal_name='Physical Review D')
    
    def articles_update(self,cites=True,institution_authors=False,institution_groups=False,DOI=False,\
                        impact_factor=False):
        self.fulldoi=pd.DataFrame()
        journal_columns=['Impact_Factor','Quartil','Journal_Hindex']
        newcolumns=['Institution_Authors','Institution_Groups','DOI','ISSN','DOI_Journal']+journal_columns
        for newcolumn in newcolumns:
            if not newcolumn in self.articles.columns:
                self.articles[newcolumn]=''

        self.articles=self.articles.reset_index(drop=True).fillna('')
        print('Updating entry:',end="")
        dbj=pd.DataFrame()
        for i in range(self.articles.shape[0]):
            time.sleep(0.3) #avoid robot detection
            print('%d.' %i,end="")
            #Update Full names: TODO: Unify algorithm
            if not self.articles.ix[i].Institution_Authors:
                institution_authors=True
                
            if self.articles.ix[i].Authors and institution_authors:
                inst_auth=''
                inst_auth_sep=';'
                for a in filter(None, self.articles.ix[0].Authors.replace('; ',';').split(';') ):
                    af=self.institution_authors[self.institution_authors.Author_Names.str.contains(a)].reset_index(drop=True)
                    if af.shape[0]==1:
                        if inst_auth:
                            inst_auth=inst_auth+inst_auth_sep
                        if af.Full_Name.values.shape[0]==1:
                            inst_auth=inst_auth+af.Full_Name.values[0]
                    elif af.shape[0]>1:
                        print('Improve real name matching')
                    if update_column(self.articles.ix[i],'Institution_Authors'):
                        self.articles.loc[i,'Institution_Authors']=inst_auth
            #Update Groups:
            if not self.articles.ix[i].Institution_Groups:
                institution_groups=True
            if self.articles.ix[i].Institution_Authors and institution_groups:
                inst_grp=''
                inst_grp_sep=';'
                chka=self.articles.ix[i].Institution_Authors.split(';')
                if len(chka)>0:
                    for fa in chka:
                        qry=self.institution_group[self.institution_group.Full_Name.str.contains(fa)].reset_index(drop=True)
                        if qry.shape[0]==1:
                            g=qry.Institution_Group.values[0]
                        else:
                            print('DEBUG improve search group for %s' %fa)
                        if inst_grp:
                            inst_grp=inst_grp+inst_grp_sep
                        
                        inst_grp=inst_grp+g
                    
                if update_column(self.articles.ix[i],'Institution_Groups'):
                        self.articles.loc[i,'Institution_Groups']=inst_grp
                    
                
            #Update DOI:
            if not self.articles.ix[i].DOI:
                rr=_get_doi(surname=self.articles.ix[i].Authors.split(';')[0].split(',')[0],\
                            title=self.articles.ix[i].Title)
                if len(rr)>0:
                    rr=pd.Series(rr)
                    if update_column(self.articles.ix[i],'DOI') and 'URL' in rr:
                        self.articles.loc[i,'DOI']=rr['URL']
                    if update_column(self.articles.ix[i],'ISSN') and 'ISSN' in rr:
                        issn=''
                        if type(rr.ISSN)==list:
                            issn=';'.join(rr.ISSN)
                        else:
                            print('DEBUG: Improve DOI -> ISSN not a list')
                        self.articles.loc[i,'ISSN']=issn
                    if update_column(self.articles.ix[i],'DOI_Journal') and 'container-title' in rr:                        
                        journal=[ j for j in rr['container-title'] if j.find('.')<0] 
                        if len(journal)==1:
                            self.articles.loc[i,'DOI_Journal']=journal[0]
                        else:
                            print('DEBUG: Improve DOI_journal have dots')
                        
                        
                self.fulldoi=self.fulldoi.append(rr,ignore_index=True).fillna('')
                
            #WARNING: Only specific journal info related code from here: See the continue !!!!
            #JOURNAL info:
            if i>0 and 'ISSN' in self.articles:
                dbj=self.articles[:i][self.articles[:i].ISSN.str.contains(\
                              self.articles.ISSN.ix[i].split(';')[0])].reset_index(drop=True)
            
            if dbj.shape[0]>0:
                for k in journal_columns:
                    if k in self.articles:
                        self.articles.loc[i,k]=dbj.ix[0][k]
                continue   
            
            #Update IF
            q=''
            if self.articles.ix[i].DOI_Journal:
                q=self.articles.ix[i].DOI_Journal.lower().replace(' ','-')
            elif self.articles.ix[i].Publication:
                q=self.articles.ix[i].Publication
            IFdf=_get_impact_factor_from_journal_name(q)
            if IFdf.shape[0]>0:
                if update_column(self.articles.ix[i],'Impact_Factor') and 'IF' in IFdf:
                    self.articles.loc[i,'Impact_Factor']=eval(IFdf.ix[0].IF)
            
            #Update quartil
            if self.articles.ix[i].ISSN.split(';')[0]:
                quartil,hindex_journal=_get_quartil(self.articles.ix[i].ISSN.split(';')[0],journal_hindex=True)
                if update_column(self.articles.ix[i],'Quartil') and  quartil:
                        self.articles.loc[i,'Quartil']=quartil
                if update_column(self.articles.ix[i],'Journal_Hindex') and hindex_journal:
                        self.articles.loc[i,'Journal_Hindex']=hindex_journal                    
            
        return self.fulldoi.fillna('')

    def to_csv(self,csvfile):
        self.articles.to_csv(csvfile,index=False)
        if self.fulldoi.shape[0]>0:
            self.fulldoi.to_json('fulldoi.json')
        
                        
            

Overwriting publications.py


In [45]:
a=articles('citations.csv',authors_file='authors.json',group_file='groups.json')

In [46]:
fulldoi=a.articles_update()

Updating entry:0.1.2.

In [43]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,5.81,Q3,70


In [33]:
self.articles.ISSN.ix[i].split(';')[0]

'1550-7998'

In [36]:
self=a
i=1
dbj=self.articles[:i][self.articles[:i].ISSN.str.contains(\
                              self.articles.ISSN.ix[i].split(';')[0])].reset_index(drop=True)

In [29]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,,,
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,,,,,,,,


In [23]:
for i in range(10):
    print(i)
    if i>2:
        continue
    print('va')

0
va
1
va
2
va
3
4
5
6
7
8
9


In [16]:
i=2
a.articles[:i]

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,,,
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,,,



Authors	Title	Publication	Volume	Number	Pages	Year	Publisher	Institution_Authors	Institution_Groups	DOI	ISSN	DOI_Journal	Impact_Factor
0	Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;	Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals	Physical Review D	79	1	13011	2009	APS	Óscar Alberto Zapata Noreña	GFIF	http://dx.doi.org/10.1103/physrevd.79.013011	1550-7998;1550-2368	Physical Review D	4.643
1	Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;	Gravitino dark matter and neutrino masses with bilinear R-parity violation	Physical Review D	85	2	23523	2012	APS	Óscar Alberto Zapata Noreña	GFIF	http://dx.doi.org/10.1103/physrevd.85.023523	1550-7998;1550-2368	Physical Review D	4.643
2	Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;	Indirect detection of gravitino dark matter including its three-body decays	Journal of Cosmology and Astroparticle Physics	2010	10	33	2010	IOP Publishing	Óscar Alberto Zapata Noreña	GFIF	http://dx.doi.org/10.1088/1475-7516/2010/10/033	1475-7516	Journal of Cosmology and Astroparticle Physic
fulldoi[:1]
fulldoi[:1]

## Save uptated data base

In [9]:
a.to_csv('newcitations.csv')

Restore saved data base

In [10]:
arts=articles('newcitations.csv',authors_file='authors.json',group_file='groups.json')

In [11]:
arts.articles_update()

Updating entry:0.1.2.

""


In [61]:
IFdf=_get_impact_factor_from_journal_name()


4.643

In [9]:
i=0
selfarticles=a.articles
if selfarticles.ix[i].DOI_Journal:
    q=selfarticles.ix[i].DOI_Journal.lower().replace(' ','-')
elif selfarticles.ix[i].Publication:
     q=selfarticles.ix[i].Publication


'http://www.journal-database.com/journal/physical-review-d.html'

In [10]:
import lxml.html as LH
import requests
import pandas as pd
def text(elt):
    return elt.text_content().replace(u'\xa0', u' ')

url = URL #'http://www.fdmbenzinpriser.dk/searchprices/5/'
r = requests.get(url)
root = LH.fromstring(r.content)

for table in root.xpath('//table'):
    header = [text(th) for th in table.xpath('//th')]        # 1
    data = [[text(td) for td in tr.xpath('td')]  
            for tr in table.xpath('//tr')]                   # 2
    data = [row for row in data if len(row)==len(header)]    # 3 
    data = pd.DataFrame(data, columns=header)                # 4
    print(data)

Empty DataFrame
Columns: []
Index: []


In [52]:
%%writefile utilities.py
#/usr/bin/env python
from bs4 import BeautifulSoup
import pandas as pd
def html_to_DataFrame(html_page_with_table,attrs={}):
    '''
    Extract the table of a web page and convert to a pandas DataFrame
    '''

    soup = BeautifulSoup(html_page_with_table,"lxml")
    table = soup.find("table",attrs)

    # The first tr contains the field names.
    headings = [th.get_text().strip() for th in table.find("tr").find_all("td")]

    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = [td.get_text() for td in row.find_all("td") if td.get_text().strip()]
        datasets.append(dataset)
        
    if headings:  
        return pd.DataFrame(datasets,columns=headings)
    else:
        return pd.DataFrame()

Writing utilities.py


In [53]:
import utilities as ut
ut.html_to_DataFrame(r.content)

,Year,IF
0,2014 (2015),4.643
1,2013,4.864
2,2012,4.691
3,2011,4.558
4,2010,4.964
5,2009,4.922
6,2008,5.050


In [38]:
if '   ': print(1)

1


In [35]:
eval(td.get_text().replace('-','0'))

TypeError: 'int' object is not callable

In [26]:
headings

['Year', 'IF']

In [12]:
from bs4 import BeautifulSoup
c=[x.text for x in BeautifulSoup(r.content,"lxml").find_all("table")]#,attrs={"id":"sortabletable"})]
print(c[0])



Year
IF


2014 (2015)
4.643


2013
4.864


2012
4.691


2011
4.558


2010
4.964


2009
4.922


2008
5.050




In [13]:
i=0
for x in BeautifulSoup(r.content,"lxml").find_all("table"):
    i=i+1
    print(i)
    x=x

1


In [14]:
x=BeautifulSoup(r.content,"lxml").find_all("table")[0]

In [125]:
import bs4

In [20]:
tx=pd.read_html( '<table>'+x.decode_contents()+'</table>',infer_types=False)[0]

/usr/lib/python3/dist-packages/pandas/io/html.py:841: FutureWarning: infer_types will have no effect in 0.14
  warnings.warn("infer_types will have no effect in 0.14", FutureWarning)


In [21]:
tx

,0,1
0,Year,IF
1,2014 (2015),4.643
2,2013,4.864
3,2012,4.691
4,2011,4.558
5,2010,4.964
6,2009,4.922
7,2008,5.050


In [106]:
x.decode_contents()

'\n<tr>\n<td>Year</td>\n<td><strong>IF</strong></td>\n</tr>\n<tr>\n<td>2014 (2015)</td>\n<td>4.643</td>\n</tr>\n<tr>\n<td>2013</td>\n<td>4.864</td>\n</tr>\n<tr>\n<td>2012</td>\n<td>4.691</td>\n</tr>\n<tr>\n<td>2011</td>\n<td>4.558</td>\n</tr>\n<tr>\n<td>2010</td>\n<td>4.964</td>\n</tr>\n<tr>\n<td>2009</td>\n<td>4.922</td>\n</tr>\n<tr>\n<td>2008</td>\n<td>5.050</td>\n</tr>\n'

In [102]:
pd.read_html?

<table>
<tr>
<td>Year</td>
<td><strong>IF</strong></td>
</tr>
<tr>
<td>2014 (2015)</td>
<td>4.643</td>
</tr>
<tr>
<td>2013</td>
<td>4.864</td>
</tr>
<tr>
<td>2012</td>
<td>4.691</td>
</tr>
<tr>
<td>2011</td>
<td>4.558</td>
</tr>
<tr>
<td>2010</td>
<td>4.964</td>
</tr>
<tr>
<td>2009</td>
<td>4.922</td>
</tr>
<tr>
<td>2008</td>
<td>5.050</td>
</tr>
</table>

In [33]:
for t in root.xpath('//table'):
    print(t)

<Element table at 0x7f075db1c4f8>


In [36]:
[text(th) for th in table.xpath('//th')]

[]

In [12]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics


In [52]:
rr=fulldoi.ix[1]
rr

DOI                                                                                                                                                                                              10.1103/physrevd.85.023523
ISSN                                                                                                                                                                                                 [1550-7998, 1550-2368]
URL                                                                                                                                                                            http://dx.doi.org/10.1103/physrevd.85.023523
article-number                                                                                                                                                                                                       023523
author              [{'affiliation': [], 'family': 'Restrepo', 'given': 'Diego'}, {'affiliation': [], 'family': 'Taoso',

In [25]:
dbj.shape

(0, 0)

In [12]:
issn='1550-7998;1550-2368'.split(';')[0]
g=_get_impact_factor_from_issn(issn)

In [15]:
g.search()

TypeError: the JSON object must be str, not 'bytes'

In [32]:
import urllib
urllib.un

In [23]:
fulldoi.append({},ignore_index=True)

,DOI,ISSN,URL,article-number,author,container-title,created,deposited,indexed,issue,issued,license,link,member,prefix,published-online,publisher,reference-count,score,source,subject,subtitle,title,type,volume,page
0,10.1103/physrevd.79.013011,"[1550-7998, 1550-2368]",http://dx.doi.org/10.1103/physrevd.79.013011,013011,"[{'affiliation': [], 'given': 'D.', 'family': 'Aristizabal Sierra'}, {'affiliation': [], 'given': 'Jisuke', 'family': 'Kubo'}, {'affiliation': [], 'given': 'Daijiro', 'family': 'Suematsu'}, {'affi...","[Physical Review D, Phys. Rev. D]","{'date-parts': [[2009, 1, 28]], 'timestamp': 1233162501000, 'date-time': '2009-01-28T17:08:21Z'}","{'date-parts': [[2015, 3, 26]], 'timestamp': 1427391551000, 'date-time': '2015-03-26T17:39:11Z'}","{'date-parts': [[2015, 12, 20]], 'timestamp': 1450639904869, 'date-time': '2015-12-20T19:31:44Z'}",1,"{'date-parts': [[2009, 1, 28]]}","[{'content-version': 'vor', 'URL': 'http://link.aps.org/licenses/aps-default-license', 'start': {'date-parts': [[2009, 1, 28]], 'timestamp': 1233100800000, 'date-time': '2009-01-28T00:00:00Z'}, 'd...","[{'intended-application': 'syndication', 'content-version': 'vor', 'URL': 'http://link.aps.org/article/10.1103/PhysRevD.79.013011', 'content-type': 'unspecified'}]",http://id.crossref.org/member/16,http://id.crossref.org/prefix/10.1103,"{'date-parts': [[2009, 1, 28]]}",American Physical Society (APS),66,1,CrossRef,[Nuclear and High Energy Physics],[],"[Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals]",journal-article,79,
1,10.1103/physrevd.85.023523,"[1550-7998, 1550-2368]",http://dx.doi.org/10.1103/physrevd.85.023523,023523,"[{'affiliation': [], 'given': 'Diego', 'family': 'Restrepo'}, {'affiliation': [], 'given': 'Marco', 'family': 'Taoso'}, {'affiliation': [], 'given': 'J. W. F.', 'family': 'Valle'}, {'affiliation':...","[Physical Review D, Phys. Rev. D]","{'date-parts': [[2012, 1, 23]], 'timestamp': 1327353600000, 'date-time': '2012-01-23T21:20:00Z'}","{'date-parts': [[2015, 3, 26]], 'timestamp': 1427349864000, 'date-time': '2015-03-26T06:04:24Z'}","{'date-parts': [[2015, 12, 20]], 'timestamp': 1450648745825, 'date-time': '2015-12-20T21:59:05Z'}",2,"{'date-parts': [[2012, 1, 23]]}","[{'content-version': 'vor', 'URL': 'http://link.aps.org/licenses/aps-default-license', 'start': {'date-parts': [[2012, 1, 23]], 'timestamp': 1327276800000, 'date-time': '2012-01-23T00:00:00Z'}, 'd...","[{'intended-application': 'syndication', 'content-version': 'vor', 'URL': 'http://link.aps.org/article/10.1103/PhysRevD.85.023523', 'content-type': 'unspecified'}]",http://id.crossref.org/member/16,http://id.crossref.org/prefix/10.1103,"{'date-parts': [[2012, 1, 23]]}",American Physical Society (APS),67,1,CrossRef,[Nuclear and High Energy Physics],[],[ Gravitino dark matter and neutrino masses with bilinear R -parity violation ],journal-article,85,
2,10.1088/1475-7516/2010/10/033,[1475-7516],http://dx.doi.org/10.1088/1475-7516/2010/10/033,,"[{'affiliation': [], 'given': 'Ki-Young', 'family': 'Choi'}, {'affiliation': [], 'given': 'Diego', 'family': 'Restrepo'}, {'affiliation': [], 'given': 'Carlos E', 'family': 'Yaguna'}, {'affiliatio...","[J. Cosmol. Astropart. Phys., Journal of Cosmology and Astroparticle Physics]","{'date-parts': [[2010, 10, 27]], 'timestamp': 1288149551000, 'date-time': '2010-10-27T03:19:11Z'}","{'date-parts': [[2011, 8, 4]], 'timestamp': 1312486993000, 'date-time': '2011-08-04T19:43:13Z'}","{'date-parts': [[2015, 12, 26]], 'timestamp': 1451141011020, 'date-time': '2015-12-26T14:43:31Z'}",10,"{'date-parts': [[2010, 10, 26]]}",,,http://id.crossref.org/member/266,http://id.crossref.org/prefix/10.1088,"{'date-parts': [[2010, 10, 26]]}",IOP Publishing,38,1,CrossRef,[Astronomy and Astrophysics],[],[Indirect detection of gravitino dark matter including its three-body decays],journal-article,2010,033-033
3,,,,,,,,,,,,,,,,,,,,,,,,,,


In [83]:
issn

'1550-7998'

('Q1', '259')

In [74]:
hindex

'259'

In [68]:
hindex

'259'